# Homework 1
## FRE-GY 7773: Machine Learning in Finance
- Fall 2021
- Due: September 24, 2021

### HW1 Notes
- Use grid search (use max depth [1-10] and return the best model), 
### Arguements
- N = 1000, d = 10

In [130]:
import pandas as pd
import numpy as np

In [131]:
df = pd.read_csv("/Users/nicktran/Desktop/Data-Analysis/ML in Finance/HW1/index_prices.csv")

In [132]:
df.set_index('Date', inplace = True) # in place means that we don't reassign back to the DF (no copying)

In [133]:
df.head()

,AGG,TLT,IEF,LQD,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY,EFA,EWJ,EWW,EWY,EZU
Date,,,,,,,,,,,,,,,,,,
9/18/2017,109.860001,126.129997,107.269997,120.900002,56.380001,66.120003,25.059999,69.779999,58.790001,55.299999,54.189999,82.570000,89.320000,68.150002,55.250000,55.910000,69.839996,43.080002
9/19/2017,109.800003,125.820000,107.150002,120.970001,56.669998,66.389999,25.250000,69.849998,59.049999,55.130001,54.080002,81.919998,89.239998,68.470001,55.570000,55.650002,70.070000,43.290001
9/20/2017,109.709999,125.910004,106.940002,120.820000,56.869999,66.839996,25.400000,70.360001,58.810001,54.599998,53.680000,82.000000,89.540001,68.320000,55.590000,55.860001,69.699997,43.080002
9/21/2017,109.690002,125.839996,106.800003,120.820000,56.750000,66.860001,25.459999,70.570000,58.490002,54.080002,53.669998,81.580002,89.389999,68.180000,55.369999,55.730000,69.690002,43.110001
9/22/2017,109.769997,126.199997,106.989998,120.980003,56.549999,67.199997,25.430000,70.750000,58.520000,53.919998,53.230000,81.580002,89.540001,68.389999,55.430000,55.990002,69.330002,43.200001


- This is a nice description of the market as a whole, utilties are dividend stocks and go up when interest rates go up. Energy is a proxy for dividends
- Build a predictive model that looks at one sector, and predicts other sectors
- When rates go down, bank stocks go 
- Risk on and risk off
- Predicting Portfolio Prices: don't do it
- Why do we do that? -> predict the returns and not the portfolio price

In [134]:
df_returns = df.pct_change(1) # this is how we see the returns on each ETF. We use df.percent_change
# This makes the first row NaN because there is no value to subtract off of

In [135]:
df_returns = df_returns[1:]

In [136]:
df_returns.head()

,AGG,TLT,IEF,LQD,XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY,EFA,EWJ,EWW,EWY,EZU
Date,,,,,,,,,,,,,,,,,,
9/19/2017,-0.000546,-0.002458,-0.001119,0.000579,0.005144,0.004083,0.007582,0.001003,0.004422,-0.003074,-0.002030,-0.007872,-0.000896,0.004696,0.005792,-0.004650,0.003293,0.004875
9/20/2017,-0.000820,0.000715,-0.001960,-0.001240,0.003529,0.006778,0.005941,0.007301,-0.004064,-0.009614,-0.007396,0.000977,0.003362,-0.002191,0.000360,0.003774,-0.005280,-0.004851
9/21/2017,-0.000182,-0.000556,-0.001309,0.000000,-0.002110,0.000299,0.002362,0.002985,-0.005441,-0.009524,-0.000186,-0.005122,-0.001675,-0.002049,-0.003958,-0.002327,-0.000143,0.000696
9/22/2017,0.000729,0.002861,0.001779,0.001324,-0.003524,0.005085,-0.001178,0.002551,0.000513,-0.002959,-0.008198,0.000000,0.001678,0.003080,0.001084,0.004665,-0.005166,0.002088
9/25/2017,0.002369,0.006260,0.002991,0.003720,0.001768,0.014435,-0.001180,0.000000,-0.011791,0.007233,0.009205,-0.002942,-0.002122,-0.005703,0.002165,-0.007144,-0.006491,-0.010880


Avoid combining the data into the same rows

### We are looking at XLF (financial sector)
- which is the target variable

In [137]:
all_cols = df.columns.values

In [138]:
all_cols

array(['AGG', 'TLT', 'IEF', 'LQD', 'XLB', 'XLE', 'XLF', 'XLI', 'XLK',
       'XLP', 'XLU', 'XLV', 'XLY', 'EFA', 'EWJ', 'EWW', 'EWY', 'EZU'],
      dtype=object)

In [139]:
cols_X = np.delete(all_cols, 6) # we are removing the target variable from the dataframe

In [140]:
df_returns_X = df_returns[cols_X]

In [141]:
df_returns_Y = df_returns['XLF']

In [142]:
df_returns_Y # Wait, we need to push the returns column down 1

Date
9/19/2017    0.007582
9/20/2017    0.005941
9/21/2017    0.002362
9/22/2017   -0.001178
9/25/2017   -0.001180
               ...   
9/10/2019    0.003589
9/11/2019    0.005723
9/12/2019    0.005334
9/13/2019    0.007782
9/16/2019   -0.003510
Name: XLF, Length: 501, dtype: float64

Now each row represents the next day's move. Notice how the last row has a NaN in it

In [143]:
df_returns_Y.shift(-1) # Now each row represents the next day's move. Notice how the last row has a NaN in it

Date
9/19/2017    0.005941
9/20/2017    0.002362
9/21/2017   -0.001178
9/22/2017   -0.001180
9/25/2017   -0.000394
               ...   
9/10/2019    0.005723
9/11/2019    0.005334
9/12/2019    0.007782
9/13/2019   -0.003510
9/16/2019         NaN
Name: XLF, Length: 501, dtype: float64

In [144]:
df_returns_Y = df_returns['XLF']

In [145]:
df_returns_Y = df_returns_Y.shift(-1)

In [146]:
df_returns_Y = df_returns_Y[:-1]
df_returns_X = df_returns_X[:-1]


In [147]:
df_returns_X.shape

(500, 17)

In [148]:
df_returns_Y.shape

(500,)

In [149]:
df_returns_Y.head()

Date
9/19/2017    0.005941
9/20/2017    0.002362
9/21/2017   -0.001178
9/22/2017   -0.001180
9/25/2017   -0.000394
Name: XLF, dtype: float64

In [150]:
df_returns_Y_binary = df_returns_Y > 0

In [151]:
df_returns_Y_binary

Date
9/19/2017     True
9/20/2017     True
9/21/2017    False
9/22/2017    False
9/25/2017    False
             ...  
9/9/2019      True
9/10/2019     True
9/11/2019     True
9/12/2019     True
9/13/2019    False
Name: XLF, Length: 500, dtype: bool

In [152]:
df_returns_Y_binary = df_returns_Y_binary.astype(int)

In [153]:
df_returns_Y_binary # our new target variable

Date
9/19/2017    1
9/20/2017    1
9/21/2017    0
9/22/2017    0
9/25/2017    0
            ..
9/9/2019     1
9/10/2019    1
9/11/2019    1
9/12/2019    1
9/13/2019    0
Name: XLF, Length: 500, dtype: int64

In [154]:
X = df_returns_X.values

In [155]:
y = df_returns_Y_binary.values

Fit DT Model

In [156]:
from sklearn.tree import DecisionTreeClassifier

In [157]:
clf10 = DecisionTreeClassifier(max_depth=10)
clf50 = DecisionTreeClassifier(max_depth=50)

In [158]:
clf.fit(X,y)

DecisionTreeClassifier()

In [159]:
y_predict = clf.predict(X)

In [160]:
from sklearn.metrics import accuracy_score, recall_score, precision_score

In [161]:
accuracy_score(y, y_predict)

1.0

# Train Test Split

In [162]:
from sklearn.model_selection import train_test_split

In [163]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.20, random_state = 42)

#### Industry Insight
Designing Our Own Splits: because we save my most recent stock data to test the model. This is because the market evolves over time and we need the most recent market data to test on. 

In [164]:
clf10.fit(X_train, y_train)
clf50.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=50)

In [165]:
print(accuracy_score(y_train, clf10.predict(X_train)))
print(accuracy_score(y_train, clf50.predict(X_train)))

0.7575
1.0


In [166]:
print(accuracy_score(y_test, clf10.predict(X_test)))
print(accuracy_score(y_test, clf50.predict(X_test)))

0.53
0.48


# Cross Validation

how many folds? 3,5,10 are good. We start getting small sectors, too large, we aren't training on enough data

In [167]:
from sklearn.model_selection import cross_val_score

In [168]:
cross_val_score(clf10, X, y, scoring=None, cv = 5) # splits the data into five sets and computes the accuracy for all of them
# we are going to take the average of these outputs

array([0.51, 0.51, 0.46, 0.4 , 0.54])

In [169]:
cross_val_score(clf50, X, y, scoring=None, cv = 5) 

array([0.37, 0.53, 0.43, 0.43, 0.56])

In [170]:
cross_val_score(clf10, X, y, scoring='average_precision', cv = 5) 

array([0.50089594, 0.52501974, 0.49072985, 0.4745667 , 0.53628579])

In [171]:
cross_val_score(clf50, X, y, scoring='average_precision', cv = 5) 

array([0.48066534, 0.52509434, 0.50369657, 0.50619846, 0.55740891])

# Grid Search CV

In [172]:
from sklearn.model_selection import GridSearchCV

In [173]:
clf = DecisionTreeClassifier()

In [187]:
grid = GridSearchCV(clf, param_grid = {'max_depth': [1,10,20,50, 100]}, cv=5) # this makes random splits and tests things that could be bad. Leakage can occur here. Where we try to predict returns based on months prior data, and not data that partains closer to the current time.
# If we try to do 5 days out: use this. Else, use autocorrelation
# Paper Trading @ Two Sigma. P and L effects. 

In [175]:
grid.fit(X,y)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [1, 10, 20, 50, 100]})

In [178]:
grid.best_estimator_

DecisionTreeClassifier(max_depth=1)

In [179]:
grid.cv_results_

{'mean_fit_time': array([0.00162444, 0.00478144, 0.00467787, 0.00445356, 0.00440035]),
 'std_fit_time': array([0.00059188, 0.00114532, 0.00068157, 0.00065763, 0.00062393]),
 'mean_score_time': array([0.00057473, 0.00035515, 0.00035696, 0.00031171, 0.00029364]),
 'std_score_time': array([1.35760203e-04, 6.12004471e-05, 4.47166487e-05, 2.61172727e-05,
        2.14595794e-05]),
 'param_max_depth': masked_array(data=[1, 10, 20, 50, 100],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 1},
  {'max_depth': 10},
  {'max_depth': 20},
  {'max_depth': 50},
  {'max_depth': 100}],
 'split0_test_score': array([0.53, 0.51, 0.37, 0.36, 0.37]),
 'split1_test_score': array([0.54, 0.49, 0.46, 0.51, 0.48]),
 'split2_test_score': array([0.53, 0.41, 0.42, 0.4 , 0.43]),
 'split3_test_score': array([0.53, 0.44, 0.42, 0.39, 0.41]),
 'split4_test_score': array([0.52, 0.55, 0.58, 0.56, 0.55]),
 'mean_test_score': array([0.53 ,

In [180]:
from sklearn.metrics import confusion_matrix

In [181]:
cm = confusion_matrix(y_test, clf50.predict(X_test))

In [182]:
cm

array([[21, 26],
       [26, 27]])

An algorithm always has an objective function that is a scalar. We maximize accuracy score, precision, recall, or F1 (average of recall and precision)